In [1]:
%matplotlib inline
%run 'bin/notebook_utils.py'
%run 'bin/plot_utilization.py'
import matplotlib.ticker as mticker
import radical.entk as re

In [2]:
! radical-stack


  python               : /Users/mturilli/.virtualenvs/analytics/bin/python3
  pythonpath           : 
  version              : 3.8.10
  virtualenv           : /Users/mturilli/.virtualenvs/analytics

  radical.analytics    : 1.5.0
  radical.entk         : 1.6.5
  radical.gtod         : 1.5.0
  radical.pilot        : 1.6.6
  radical.saga         : 1.6.6
  radical.utils        : 1.6.6



In [3]:
suds = ['re.session.login1.lei.018775.0008']
sdir = '../data/raw/incite2021/'
sids = [sdir+s for s in suds]

In [4]:
%%capture capt

ss = {}
for sid in suds:
    sp = sdir+sid
    ss[sid] = {'s': ra.Session(sp, 'radical.pilot')}
    ss[sid].update({'p': ss[sid]['s'].filter(etype='pilot'   , inplace=False),
                    't': ss[sid]['s'].filter(etype='task'    , inplace=False),
                    'w': ss[sid]['s'].filter(etype='pipeline', inplace=False)})

In [5]:
for sid in suds:
    ss[sid].update({'sid'     : ss[sid]['s'].uid,
                    'pid'     : ss[sid]['p'].list('uid'),
                    'npilot'  : len(ss[sid]['p'].get()),
                    'lm'      : ss[sid]['s'].get(etype='pilot')[0].cfg['agent_launch_method'],
                    'npact'   : len(ss[sid]['p'].timestamps(state='PMGR_ACTIVE')),
                    'nunit'   : len(ss[sid]['t'].get()),
                    'nudone'  : len(ss[sid]['t'].timestamps(state='DONE')),
                    'nufail'  : len(ss[sid]['t'].timestamps(state='FAILED')),
                    'hostid'  : ss[sid]['s'].get(etype='pilot')[0].cfg['hostid'],
                    'gpu_node': ss[sid]['s'].get(etype='pilot')[0].cfg['gpus_per_node']})
    ss[sid].update({'pres'    : ss[sid]['p'].get(uid=ss[sid]['pid'])[0].description['resource'],
                    'ncores'  : ss[sid]['p'].get(uid=ss[sid]['pid'])[0].description['cores'],
                    'ngpus'   : ss[sid]['p'].get(uid=ss[sid]['pid'])[0].description['gpus']})
    ss[sid].update({'nnodes'  : int(ss[sid]['ngpus']/ss[sid]['gpu_node'])})

In [6]:
info = []
for sid in suds:
    info.append({'session'  : sid,
                 'resource' : ss[sid]['pres'],
                 'pilot_lm' : ss[sid]['lm'], 
                 'pilots'   : ss[sid]['npilot'],
                 'ps_active': ss[sid]['npact'],
                 'cores'    : ss[sid]['ncores']/4, 
                 'gpus'     : ss[sid]['ngpus'], 
                 'nodes'    : ss[sid]['nnodes'], 
                 'tasks'    : ss[sid]['nunit'], 
                 'ts_done'  : ss[sid]['nudone'],  
                 'ts_failed': ss[sid]['nufail']})

df_info = pd.DataFrame(info) 
df_info

,session,resource,pilot_lm,pilots,ps_active,cores,gpus,nodes,tasks,ts_done,ts_failed
0,re.session.login1.lei.018775.0008,ornl.summit,JSRUN,1,1,2688.0,384,64,2,2,0


In [7]:
for sid in suds:
    w  = ss[sid]['w']
    t  = ss[sid]['t']
    p  = ss[sid]['p']
    p0 = p.get(uid=ss[sid]['pid'])[0]

    ss[sid].update({
      'ttc'  : p0.duration(event=[ {ru.EVENT: 'state'        , ru.STATE: rp.NEW                }, 
                                  [{ru.EVENT: 'state'        , ru.STATE: rp.DONE               },
                                   {ru.EVENT: 'state'        , ru.STATE: rp.CANCELED           },
                                   {ru.EVENT: 'state'        , ru.STATE: rp.FAILED             }]]),
      'ttq_p': p0.duration(event=[ {ru.EVENT: 'state'        , ru.STATE: rp.PMGR_ACTIVE_PENDING}, 
                                   {ru.EVENT: 'state'        , ru.STATE: rp.PMGR_ACTIVE        } ]),
      'ttx_w': w.duration( event=[ {ru.EVENT: 'state'        , ru.STATE: re.states.SCHEDULING  },
                                   {ru.EVENT: 'state'        , ru.STATE: re.states.DONE        } ]),
      'ttx_u': t.duration( event=[ {ru.EVENT: 'task_exec_start', ru.STATE: None                  },      
                                   {ru.EVENT: 'task_exec_stop' , ru.STATE: None                  } ]),
      'ttx_p': p0.duration(event=[ {ru.EVENT: 'state'        , ru.STATE: rp.PMGR_ACTIVE        },
                                  [{ru.EVENT: 'state'        , ru.STATE: rp.DONE               },
                                   {ru.EVENT: 'state'        , ru.STATE: rp.FAILED             },
                                   {ru.EVENT: 'state'        , ru.STATE: rp.CANCELED           }]]),
      'sub_t': t.duration( event=[ {ru.EVENT: 'state'        , ru.STATE: re.states.SCHEDULING  },
                                   {ru.EVENT: 'state'        , ru.STATE: re.states.SUBMITTING  } ]),
      'ttx_t': t.duration( event=[ {ru.EVENT: 'state'        , ru.STATE: re.states.SUBMITTING  },
                                  [{ru.EVENT: 'state'        , ru.STATE: re.states.COMPLETED   },
                                   {ru.EVENT: 'state'        , ru.STATE: re.states.CANCELED    },
                                   {ru.EVENT: 'state'        , ru.STATE: re.states.FAILED      }]])})
    
    ss[sid].update({'ovh_rtc': ss[sid]['ttx_w'] - ss[sid]['ttx_u']})

In [8]:
skip = []
nsids = len(suds) - len(skip)

fwidth = 6*nsids
fhight = 4
fig, axarr = plt.subplots(1, nsids, sharey=True, figsize=(fwidth, fhight))

i = 0
j = 'a'
for sid in suds:

    if sid in skip:
        continue

    if len(suds) > 1:
        ax = axarr[i]
    else:
        ax = axarr
    
    ax.title.set_text('%s/%s Tasks/Nodes' % (ss[sid]['nunit'], int(ss[sid]['nnodes'])))

    ax.bar(x = 'OVH', height = ss[sid]['ovh_rtc'], color = tableau20[1])
    ax.bar(x = 'TTX', height = ss[sid]['ttx_t']  , color = tableau20[3])

    ax.set_xlabel('(%s)' % j, labelpad=45)

    i = i+1
    j = chr(ord(j) + 1)

fig.text( -0.05,  0.5 , 'Time (s)', va='center', rotation='vertical', fontsize=24)
fig.text(  0.5 , -0.05, 'Metric'  , ha='center', fontsize=24)
fig.legend(['RADICAL Cybertools overhead (OVH)', 'Workflow time to completion (TTX)'], loc='upper center', 
           bbox_to_anchor=(0.52, 1.35), fontsize=24, ncol=1)
    
plt.savefig('figures/incite_2021_ovh_ttx.pdf', dpi=300, bbox_inches='tight')
plt.savefig('figures/incite_2021_ovh_ttx.png', dpi=300, bbox_inches='tight')

In [9]:
metrics_default = [

        # pilot overheads (global)
        #['Pilot Startup',     ['boot', 'setup_1']],
        #['Pilot Termination', ['term' ]],

        # derived pilot durations
        #['Agent Nodes',       ['agent']],
        #['Warmup',            ['warm' ]],
        #['Draining',          ['drain']],
        #['Idle',              ['idle' ]],

        # unit durations (PRTE)
        #['Prepare Exec',      ['exec_queue', 'exec_prep']],
        #['Exec RP',           ['exec_rp', 'exec_sh', 'term_sh', 'term_rp']],
        ['Exec Cmd',          ['exec_cmd']],
        #['Unschedule',        ['unschedule']],
]

In [10]:
%%capture capt

# Remove cache to avoid reloading errors
! rm -r ~/.radical/analytics

# Skipe sessions we do not need to plot
skip = []
sids_pruned = np.setdiff1d(sids, skip)

sobjs = ra.Experiment(sids_pruned, stype='radical.pilot')
provided, consumed, stats_abs, stats_rel, info = sobjs.utilization(metrics=metrics_default)

In [11]:
# x-axis labels
xkeys = list()

# get the numbers we actually want to plot
for session in sobjs.sessions:

    n_units = len(session.get(etype='task'))
    
    p_size = 0
    for pilot in session.get(etype='pilot'):
        p_size += pilot.description['cores']

    xkeys.append('%s / %s' % (n_units, p_size))

# zero points to zero the axes of the plot.
p_zeros = {}
for session in sobjs.sessions:
    for pilot in session.get(etype='pilot'):
        p_zeros[session.uid] = pilot.timestamps(event={ru.EVENT: 'bootstrap_0_start'})[0]

def update_ticks(x, pos):
    return int(x/168)

In [12]:
# # core/gpu utilization over time (box plot)

# fwidth = 36
# fhight = 4
# fig, axarr = plt.subplots(1, len(suds), sharex='col', figsize=(fwidth, fhight))  # , sharey='row')

# i = 0
# j = 'a'
# for sid in suds:

#     ax = axarr[i]
#     cmap = mpl.cm.get_cmap('tab20c')

#     step = 1.0 / (len(metrics_default) + 1)
#     this = step / 1.0
#     legend = list()

#     x_min = None
#     x_max = None
#     y_min = None
#     y_max = None
    
#     ax.set_title('%s/%s Tasks/Nodes' % (ss[sid]['nunit'], int(ss[sid]['nnodes'])), fontdict={'fontsize': 30})

#     for metric in metrics_default:

#         color = cmap(this)
#         this += step

#         legend.append(mpl.lines.Line2D([0], [0], color=color, lw=6))

#         if isinstance(metric, list):
#             name = metric[0]
#             parts = metric[1]
#         else:
#             name = metric
#             parts = [metric]

#         for part in parts:
#             for uid in consumed[sid][part]:
#                 for block in consumed[sid][part][uid]:
#                     orig_x = block[0]
#                     orig_y = block[2] - 0.5
#                     width = block[1] - block[0]
#                     height = block[3] - block[2] + 1.0

#                     if x_min is None:
#                         x_min = orig_x
#                     if x_max is None:
#                         x_max = orig_x + width
#                     if y_min is None:
#                         y_min = orig_x
#                     if y_max is None:
#                         y_max = orig_x + height

#                     x_min = min(x_min, orig_x)
#                     y_min = min(y_min, orig_y)
#                     x_max = max(x_max, orig_x + width)
#                     y_max = max(y_max, orig_y + height)

#                     patch = mpl.patches.Rectangle((orig_x, orig_y),
#                                                   width, height,
#                                                   facecolor=color,
#                                                   edgecolor='black',
#                                                   fill=True, lw=0.0)
#                     ax.add_patch(patch)

#     ax.set_xlim([x_min, x_max])
#     ax.set_ylim([y_min, y_max])
    
#     ax.set_xlabel('(%s)' % j, labelpad=45)
#     ax.yaxis.set_major_formatter(mticker.FuncFormatter(update_ticks))

#     i = i+1
#     j = chr(ord(j) + 1)

# # Add common axes lables
# fig.text(0.07,  0.5, 'Core ID' , va='center', rotation='vertical', fontsize=30)
# fig.text(0.5 , -0.1, 'Time (s)', ha='center', fontsize=30)
    
# # Add legend
# fig.legend(legend, [m[0] for m in metrics_default], loc='upper center', 
#            bbox_to_anchor=(0.45, 1.77), fontsize=30, ncol=4)

# plt.savefig('figures/incite2021_inversions_jsrun.pdf', dpi=300, bbox_inches='tight')
# plt.savefig('figures/incite2021_inversions_jsrun.png', dpi=300, bbox_inches='tight')

splot = ['re.session.login1.lei.018775.0008']

nsids = len(splot)

fwidth = 12*nsids
fhight = 8
fig, axarr = plt.subplots(1, nsids, sharex='col', figsize=(fwidth, fhight))

i = 0
j = 'a'
for sid in splot:
    if nsids > 1:
        ax = axarr[i]
        ax.set_xlabel('(%s)' % j, labelpad=45)
    else:
        ax = axarr

    cmap = mpl.cm.get_cmap('tab20c')

    step = 1.0 / (len(metrics_default) + 1)
    this = step / 1.0
    legend = list()

    x_min = None
    x_max = None
    y_min = None
    y_max = None
        
    ax.set_title(
        '%s/%s Tasks/Nodes' % (
            ss[sid]['nunit'], 
            int(ss[sid]['nnodes'])), 
        fontdict={'fontsize': 30})

    for metric in metrics_default:

        color = tableau20[8] #cmap(this)
        this += step

        legend.append(mpl.lines.Line2D([0], [0], color=color, lw=6))

        if isinstance(metric, list):
            name = metric[0]
            parts = metric[1]
        else:
            name = metric
            parts = [metric]

        for part in parts:
            for uid in consumed[sid][part]:
                for block in consumed[sid][part][uid]:
                    orig_x = block[0] - p_zeros[sid]
                    orig_y = block[2] - 0.5
                    width = block[1] - block[0]
                    height = block[3] - block[2] + 1.0

                    if x_min is None:
                        x_min = orig_x
                    if x_max is None:
                        x_max = orig_x + width
                    if y_min is None:
                        y_min = orig_x
                    if y_max is None:
                        y_max = orig_x + height

                    x_min = min(x_min, orig_x)
                    y_min = min(y_min, orig_y)
                    x_max = max(x_max, orig_x + width)
                    y_max = max(y_max, orig_y + height)

                    patch = mpl.patches.Rectangle((orig_x, orig_y),
                                                  width, height,
                                                  facecolor=color,
                                                  edgecolor='black',
                                                  fill=True, lw=0.0)
                    ax.add_patch(patch)

    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    
    # ax.set_xlabel('Time (s)', fontsize=30)
    # ax.set_ylabel('Node', fontsize=30)
    
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(update_ticks))

    i = i+1
    j = chr(ord(j) + 1)
    
# Add legend
# ax.legend(legend, [m[0] for m in metrics_default], loc='upper center', fontsize=30, ncol=4, bbox_to_anchor=( -0.5, 1.35))
fig.legend(['Task execution time on CPU cores or GPUs'], loc='upper center', bbox_to_anchor=(0.5, 1.05), fontsize=24, ncol=1)
fig.text( 0.05,  0.5 , 'Node'    , va='center', rotation='vertical', fontsize=30)
fig.text( 0.5 ,  0.02, 'Time (s)', ha='center', fontsize=30)

plt.savefig('figures/incite_2021_ru.pdf', dpi=300, bbox_inches='tight')
plt.savefig('figures/incite_2021_ru.png', dpi=300, bbox_inches='tight')